# Autograd implementation

In a neural network, we pass the input through a network layer and calculate the output of the layer straightforwardly. This step is called forward-propagation. Each layer also implements a function called 'backward'. Backward is responsible for the backward pass of back-propagation. The process of back-propagation follows the schemas: Input -> Forward calls -> Loss function -> derivative -> back-propagation of errors. In a neural network, any layer can forward its results to many other layers, in this case, in order to do back-propagation, we sum the deltas coming from all the target layers. 

In [ ]:
'''backprop implementation with layer abstraction.'''
import numpy as np

class Layer:
    '''A layer in a network.

    A layer is simply a function from R^n to R^d for some specified n and d.
    A neural network can usually be written as a sequence of layers:
    if the original input x is in R^d, a 3 layer neural network might be:

    L3(L2(L1(x)))

    We can also view the loss function as itself a layer, so that the loss
    of the network is:

    Loss(L3(L2(L1(x))))

    This class is a base class used to represent different kinds of layer
    functions. We will eventually specify a neural network and its loss function
    with a list:

    [L1, L2, L3, Loss]

    where L1, L2, L3, Loss are all Layer objects.

    Each Layer object implements a function called 'forward'. forward simply
    computes the output of a layer given its input. So instead of
    Loss(L3(L2(L1(x))), we write
    Loss.forward(L3.forward(L2.forward(L1.forward(x)))).
    Doing this computation finishes the forward pass of backprop.

    Each layer also implements a function called 'backward'. Backward is
    responsible for the backward pass of backprop. After we have computed the
    forward pass, we compute
    L1.backward(L2.backward(L3.backward(Loss.backward(1))))
    We give 1 as the input to Loss.backward because backward is implementing
    the chain rule - it multiplies gradients together and so giving 1 as an
    input makes the multiplication an identity operation.

    The outputs of backward are a little subtle. Some layers may have a
    parameter that specifies the function being computed by the layer. For
    example, a Linear layer maintains a weight matrix, so that
    Linear(x) = xW for some matrix W.
    
    The input to backward should be the gradient of the final loss with respect
    to the output of the current layer. The output of backward should be the
    gradient of the final loss with respect to the input of the current layer,
    which is just the output of the previous layer. This is why it is correct
    to chain the outputs of backprop together. However, backward should ALSO
    compute the gradient of the loss with respect to the current layer's
    parameter and store this internally to be used in training.
    '''
    def __init__(self, parameter=None, name=None):
        self.name = name
        self.forward_called = False
        self.parameter = parameter
        self.grad = None

    def zero_grad(self):
        self.grad = None

    def forward(self, input):
        '''forward pass. Should compute layer and save relevant state
        needed for backward pass.
        Args:
            input: input to this layer.
        returns output of operation.
        '''
        raise NotImplementedError

    def backward(self, downstream_grad):
        '''Performs backward pass.

        This function should also set self.grad to be the gradient of the final
        output of the computation with respect to the parameter.

        Args:
            downstream_grad: gradient from downstream operation in the
                computation graph. This package will only consider
                computation graphs that result in scalar outputs at the final
                node (e.g. loss function computations). As a result,
                the dimension of downstream_grad should match the dimension of
                the output of this layer.

                Formally, if this operation computes F(x), and the final
                computation computes a scalar, G(F(x)), then input_grad is
                dG/dF.
        returns:
            gradient to pass to upstream layers. If the layer computes F(x, w),
            where x is the input and w is the parameter of the layer, then
            the return value should be dF(x,w)/dx * downstream_grad. Here,
            x is in R^n, F(x, w) is in R^m, dF(x, w)/dx is a matrix in R^(n x m)
            downstream_grad is in R^m and * indicates matrix multiplication.

        We should also compute the gradient with respect to the parameter w.
        Again by chain rule, this is dF(x, w)/dw * downstream_grad
        '''
        raise NotImplementedError

Below shows an example of the full implementation of a bias layer, including the forward and backward function. Notice self.grad stores the gradient of the loss with respect to the current layer's parameter.

In [ ]:
class Bias(Layer):
    '''adds a constant bias.
    The bias is assumed to be a K-dimensional ndarray for arbitary K that is
    added to the last K dimensions of a D dimensional input and broadcast
    over the first D-K dimensions.'''

    def __init__(self, bias, name="bias"):
        super(Bias, self).__init__(np.squeeze(bias), name)
        self.weights = np.squeeze(bias)

    def forward(self, input):
        self.input = input
        return self.parameter + self.input

    def backward(self, downstream_grad):
        self.grad = np.sum(downstream_grad, tuple(range(downstream_grad.ndim - self.parameter.ndim)))
        return downstream_grad

## Multiplication layer.

Let's look at the basic linear layer.

$Z_{linear} = XW$



Assuming that loss $L$ is a function of $Z$, What is $\frac{\partial{L}}{\partial{X}}$ ?

$Z$, $X$ and $W$ are each $2$-tensors, therefore the derivative of any one of them w.r.t. any other will be a $4$-tensor. $Z$ has size $(B \times M)$, $X$ has size $(B \times N)$, and $W$ has size $(N \times M)$.

Focusing on the $\{i,j\}$ element of $Z$:

$$
Z_{ij} = \sum_{k=1}^B X_{ik} W_{kj}
$$

$\partial{Z_{ij}}/\partial{X}$ is a $(B \times N)$ matrix, but is only nonzero in row $i$, where it's values are the $j$th column of $W$. Therefore the full derivative of $Z$ w.r.t. $X$ is a sparse Jacobian defined by:

$$
\boxed{\left(\frac{\partial{Z}}{\partial{X}}\right)_{ijik} = W_{kj}}
$$

with all other values equal to zero.

When applying the chain rule for backpropagation from a downstream function $L(Z)$, we do not need to compute the full jacobian $\partial{Z}/\partial{X}$. Instead, we get:

$$
\frac{\partial{L}}{\partial{X}} = \frac{\partial{L}}{\partial{Z}}\frac{\partial{Z}}{\partial{X}} = \frac{\partial{L}}{\partial{Z}} W^T
$$

This is merely a convenient rearrangement of the terms of $\partial{Z}/\partial{X}$.

Using this, we can complete the forward and backward function of the linear layer. In backward, we ALSO set the self.grad to be the gradient of the loss with respect to the current layer's parameter.

In [ ]:
class Linear(Layer):
    '''Linear layer. Parameter is NxM matrix L, input is matrix v of size B x N
    where B is batch size, output is vL.'''

    def __init__(self, weights, name="Linear"):
        super(Linear, self).__init__(weights, name)
        self.parameter = weights

    def forward(self, input):
        self.input = input
        return self.input @ self.parameter

    def backward(self, downstream_grad):
        '''downstream_grad should be BxM.'''
        self.grad = np.transpose(self.input) @ downstream_grad #dL/dW = X' * dL/dZ
        return downstream_grad @ np.transpose(self.parameter) #dL/dX = dL/dZ * W'

## Activation layers.

Now let's look at a popular activation layer:

$ReLU(x) = max(0,x)$


$ReLU(x)$ Derivative:

$$
ReLU(x)
= \left\{ \begin{array}{rcl}
0 & \mbox{for} & x \le 0 \\
x & \mbox{for} & x > 0
\end{array}\right.
$$

$$
\boxed{\frac{\partial}{\partial{x}}ReLU(x)
= \left\{ \begin{array}{rcl}
0 & \mbox{for} & x \le 0 \\
1 & \mbox{for} & x > 0
\end{array}\right.}
$$

Now we can write the forward and backward functions for this layer. There is no need to update self.grad since there is no parameter in activation layers. 

In [ ]:
class ReLU(Layer):
    '''ReLU layer. No parameters.'''

    def __init__(self, name="ReLU"):
        super(ReLU, self).__init__(name=name)

    def forward(self, input):
        self.input = input
        return np.maximum(0,self.input)

    def backward(self, downstream_grad):
        upstream_grad = downstream_grad.copy()
        upstream_grad[self.input <= 0] = 0;
        return upstream_grad

##Another Example: Softmax

$\sigma(j)=\frac{\exp(w_j^Tx)}{\sum_{k}\exp(w_k^Tx)}$.

When $i = j$,

$$
\frac{\partial}{\partial{x_i}}\sigma(j) \ = \ w_i \sigma(i)(1-\sigma(i))
$$

When $i \ne j$,
$$
\frac{\partial}{\partial{x_i}}\sigma(j) \ = -w_i\sigma(i)\sigma(j)
$$

Now we can write the forward and backward functions for a softmax layer. There is no need to update self.grad since there is no parameter in a softmax function.

In [ ]:
class SoftMax(Layer):
    '''SoftMax Layer, no parameters.'''

    def __init__(self, name="SoftMax"):
        super(SoftMax, self).__init__(name="softmax")

    def forward(self, input):
        '''input is BxN array, B is batch dimension.'''
        e = np.exp(input - np.max(input)) #max over entire matrix
        self.p = e / np.sum(e, axis=1)[:,None]
        return self.p

    def backward(self, downstream_grad):
        '''downstream grad should be BxN array.'''
        return self.p * (downstream_grad - np.sum(downstream_grad * self.p, axis=1)[:,None])

... And so on.